# Mini-Projeto - Analisando Dados do Uber com Spark 

Dataset Original: https://github.com/fivethirtyeight/uber-tlc-foil-response

O conjunto de dados original contém dados de mais de 4,5 milhões de captações Uber na cidade de Nova York de abril a setembro de 2014 e 14,3 milhões de captações Uber de janeiro a junho de 2015. Dados em nível de viagem sobre 10 outras empresas de veículos de aluguel (FHV) bem como dados agregados para 329 empresas de FHV, também estão incluídos. Todos os arquivos foram recebidos em 3 de agosto, 15 de setembro e 22 de setembro de 2015.

#### A análise realizada nesse notebook será feita em um conjunto de dados reduzido para demonstração e aprimoramento dos conhecimentos.

1- Quantos são e quais são as bases de carros do Uber (onde os carros ficam esperando passageiros)?

2- Qual o total de veículos que passaram pela base B02617?

3- Qual o total de corridas por base? Apresente de forma decrescente.

###### Etapa 1. Carregando os dados utilizando Pandas e realizando uma análise dos dados existentes.

In [1]:
from pandas import read_csv

In [2]:
# Criando o objeto
uber_dataset = read_csv("/data/uber.csv")

In [3]:
# Verificando o tipo do objeto
type(uber_dataset)

pandas.core.frame.DataFrame

In [4]:
uber_dataset.head(15)

,dispatching_base_number,date,active_vehicles,trips
0,B02512,1/1/2015,190,1132
1,B02765,1/1/2015,225,1765
2,B02764,1/1/2015,3427,29421
3,B02682,1/1/2015,945,7679
4,B02617,1/1/2015,1228,9537
5,B02598,1/1/2015,870,6903
6,B02598,1/2/2015,785,4768
7,B02617,1/2/2015,1137,7065
8,B02512,1/2/2015,175,875
9,B02682,1/2/2015,890,5506


###### Etapa 2: Transformando o dataframe do Pandas em dataframe do Spark

In [5]:
uber_dataset_spk = sqlContext.createDataFrame(uber_dataset)

In [6]:
type(uber_dataset_spk)

pyspark.sql.dataframe.DataFrame

###### Extra: Transformando um Dataframe Pandas e Dataframe Spark em RDD's

In [7]:
# Transformando um dataframe do pandas em RDD com a função parallelize do Spark Context
df_pandas_rdd = sc.parallelize(uber_dataset)
type(df_pandas_rdd)

pyspark.rdd.RDD

In [8]:
# Transformando um dataframe do Spark em RDD através da Operação RDD.
df_spark_rdd = uber_dataset_spk.rdd
type(df_spark_rdd)

pyspark.rdd.RDD

###### Etapa 1 e 2 Extra: Realizando a mesma importação direta com o Spark resultando em um RDD
###### Com 1 linha de código realizamos a mesma importação.

In [9]:
uberRDD = sc.textFile("~/data/uber.csv")
type(uberRDD)

pyspark.rdd.RDD

In [10]:
uberRDD.take(15)

['dispatching_base_number,date,active_vehicles,trips',
 'B02512,1/1/2015,190,1132',
 'B02765,1/1/2015,225,1765',
 'B02764,1/1/2015,3427,29421',
 'B02682,1/1/2015,945,7679',
 'B02617,1/1/2015,1228,9537',
 'B02598,1/1/2015,870,6903',
 'B02598,1/2/2015,785,4768',
 'B02617,1/2/2015,1137,7065',
 'B02512,1/2/2015,175,875',
 'B02682,1/2/2015,890,5506',
 'B02765,1/2/2015,196,1001',
 'B02764,1/2/2015,3147,19974',
 'B02765,1/3/2015,201,1526',
 'B02617,1/3/2015,1188,10664']

In [11]:
uberRDD.count()

355

In [12]:
# Removendo o Header do RDD para facilitar durante a manipulação dos dados
uberRDD_header = uberRDD.first()
uberRDD_no_header = uberRDD.filter(lambda x : x != uberRDD_header)
uberRDD_no_header.take(3)

['B02512,1/1/2015,190,1132',
 'B02765,1/1/2015,225,1765',
 'B02764,1/1/2015,3427,29421']

In [13]:
# Realizando o split do arquivo RDD a cada vez que encontrar o caracter ',' facilitando assim análises futuras
# A transformação executada através da operação map, irá resultar em um pipelineRDD.
uberRDD_splitted = uberRDD_no_header.map(lambda line: line.split(","))
uberRDD_splitted.take(10)

[['B02512', '1/1/2015', '190', '1132'],
 ['B02765', '1/1/2015', '225', '1765'],
 ['B02764', '1/1/2015', '3427', '29421'],
 ['B02682', '1/1/2015', '945', '7679'],
 ['B02617', '1/1/2015', '1228', '9537'],
 ['B02598', '1/1/2015', '870', '6903'],
 ['B02598', '1/2/2015', '785', '4768'],
 ['B02617', '1/2/2015', '1137', '7065'],
 ['B02512', '1/2/2015', '175', '875'],
 ['B02682', '1/2/2015', '890', '5506']]

In [14]:
type(uberRDD_splitted)

pyspark.rdd.PipelinedRDD

###### 1- Quantos são e quais são as bases de carros do Uber (onde os carros ficam esperando passageiros)?

In [15]:
# Quantidade de bases de carros distintos da Uber
uberRDD_splitted.map(lambda x: x[0]).distinct().count()

6

In [16]:
# Nome das bases da uber
uberRDD_splitted.map(lambda x: x[0]).distinct().collect()

['B02765', 'B02682', 'B02598', 'B02512', 'B02764', 'B02617']

###### 2- Qual o total de veículos que passaram pela base B02617?

In [17]:
uberRDD_splitted.filter(lambda x: 'B02617' in x[0]).count()

59

###### 3- Qual o total de corridas por base? Apresente de forma decrescente.

In [18]:
uberRDD_splitted.map(lambda x: (x[0],int(x[3]))).reduceByKey(lambda acc, n: acc + n).sortBy(lambda a: -a[1]).collect()

[('B02764', 1914449),
 ('B02617', 725025),
 ('B02682', 662509),
 ('B02598', 540791),
 ('B02765', 193670),
 ('B02512', 93786)]